In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQshap7jt9b6sowfrbO9wo6Xkd6mFPl3V0TPA&usqp=CAU',width=400,height=400)

indianpersian.com

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
from plotly.offline import iplot

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df= pd.read_csv('/kaggle/input/persian-books-dataset/data.csv',error_bad_lines = False)
df.head()

#Codes from Niharika Pandit https://www.kaggle.com/niharika41298/netflix-vs-books-recommender-analysis-eda  The resize I got from #https://www.computerhope.com/issues/ch000055.htm
<width="200" height="40" alt="Computer Hope">

In [ ]:
joint_df=pd.merge(df,df,left_on='book_id',right_on='book_id',how='inner')

In [ ]:
#Ascending order False
top_rated=df.sort_values('rating', ascending=False)
top10=top_rated.head(10)
f=['title','coveruri']
displ=(top10[f])
displ.set_index('title', inplace=True)

In [ ]:
from IPython.display import Image, HTML

def path_to_image_html(path):
    '''
     This function essentially convert the image url to 
     '<img src="'+ path + '"/>' format. And one can put any
     formatting adjustments to control the height, aspect ratio, size etc.
     within as in the below example. 
    '''

    return '<img src="'+ path + '""/width="200" height="40" alt="Computer Hope">'

HTML(displ.to_html(escape=False ,formatters=dict(coveruri=path_to_image_html),justify='center'))

In [ ]:
#Ascending order True
pop10=df.sort_values(by='rating', ascending=True)
f=['title','coveruri']
pop10=pop10.head(10)

pop10=(pop10[f])
pop10=pop10.set_index('title')

In [ ]:
HTML(pop10.to_html(escape=False ,formatters=dict(coveruri=path_to_image_html),justify='center'))

#Most Common Rating Values

In [ ]:
import seaborn as sns
plt.figure(figsize=(10,6))
sns.distplot(a=df['rating'], kde=True, color='r')

As we can read the distplot above, the most common rating is somewhere between 3 to 5.

In [ ]:
no_of_ratings_per_book=ratings.groupby('book_id').count()

In [ ]:
no_of_ratings_per_book

In [ ]:
plt.figure(figsize=(10,4))
sns.distplot(a=no_of_ratings_per_book['rating'], color='g')

Some books have ratings less than 10. Other books got ratings greater than 80.

In [ ]:
f=['author_name', 'rating']
top_author_name=top_rated[f]
top_author_name=top_author_name.head(20)

In [ ]:
fig = px.bar(top_author_name, x='author_name', y='rating', color ='rating')
fig.show()

In [ ]:
cnt_srs = df['categories'].value_counts().head()
trace = go.Bar(
    y=cnt_srs.index[::-1],
    x=cnt_srs.values[::-1],
    orientation = 'h',
    marker=dict(
        color=cnt_srs.values[::-1],
        colorscale = 'Greens',
        reversescale = True
    ),
)

layout = dict(
    title='Categories Distribution',
    )
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="categories")

In [ ]:
fillnadf= df.fillna('')

In [ ]:
def clean_data(x):
        return str.lower(x.replace(" ", ""))

In [ ]:
features=['title','author_name','rating']
fillednadf=fillnadf[features]

In [ ]:
fillednadf = fillednadf.astype(str)
fillednadf.dtypes

In [ ]:
for feature in features:
    fillednadf[feature] = fillednadf[feature].apply(clean_data)
    
fillednadf.head(2)

#Creating a "soup" or a "bag of words" for all rows.

In [ ]:
def create_soup(x):
    return x['title']+ ' ' + x['author_name'] + ' ' + x['rating']

In [ ]:
fillednadf['soup'] = fillednadf.apply(create_soup, axis=1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(fillednadf['soup'])

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
fillednadf=fillednadf.reset_index()
indices = pd.Series(fillednadf.index, index=fillednadf['title'])

In [ ]:
def get_recommendations_new(title, cosine_sim=cosine_sim2):
    title=title.replace(' ','').lower()
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return list(df['title'].iloc[movie_indices])

In [ ]:
l=get_recommendations_new('The Prince', cosine_sim2)
fig = go.Figure(data=[go.Table(header=dict(values=l,fill_color='orange'))
                     ])
fig.show()

In [ ]:
l=get_recommendations_new('مرد عنکبوتی - ددپول (۲۲)', cosine_sim2)
fig = go.Figure(data=[go.Table(header=dict(values=l,fill_color='orange'))
                     ])
fig.show()

In [ ]:
usecols=['book_id', 'title']
df_col=df[usecols]

 Dropping null values.

In [ ]:
df_col.dropna()

Das War's, Kaggle Notebook Runner: Marília Prata  @mpwolke